## Decentralised Load Forecasting with Graph Neural Networks and Edge AI

Name: Kevin Fastje  
Student ID: a1933491

### Assignment 2: Part B – Big Data Analysis

In [15]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

#### 1. Loading Data

In [8]:
demand_df = pd.read_csv('/Users/kevin/OneDrive - University of Adelaide/MAIML/7209 Big Data Analysis and Project/Robust-Capacity-Expansion-main/InputData/Demand/El_demand_2050_DE_processed.csv')
print(demand_df.shape)
print(demand_df.columns)
print(demand_df.head(7))

(8760, 39)
Index(['Unnamed: 0', 'DE11', 'DE12', 'DE13', 'DE14', 'DE21', 'DE22', 'DE23',
       'DE24', 'DE25', 'DE26', 'DE27', 'DE30', 'DE40', 'DE50', 'DE60', 'DE71',
       'DE72', 'DE73', 'DE80', 'DE91', 'DE92', 'DE93', 'DE94', 'DEA1', 'DEA2',
       'DEA3', 'DEA4', 'DEA5', 'DEB1', 'DEB2', 'DEB3', 'DEC0', 'DED2', 'DED4',
       'DED5', 'DEE0', 'DEF0', 'DEG0'],
      dtype='object')
   Unnamed: 0   DE11   DE12   DE13   DE14   DE21   DE22   DE23   DE24   DE25  \
0           0  3.237  2.329  1.884  1.546  4.780  1.108  0.992  0.768  1.336   
1           1  3.189  2.289  1.852  1.532  4.704  1.091  0.981  0.761  1.300   
2           2  3.101  2.234  1.803  1.495  4.589  1.068  0.958  0.738  1.256   
3           3  3.059  2.214  1.784  1.477  4.549  1.058  0.946  0.727  1.236   
4           4  3.042  2.209  1.766  1.468  4.511  1.045  0.933  0.716  1.217   
5           5  3.127  2.269  1.789  1.504  4.625  1.073  0.953  0.732  1.245   
6           6  3.178  2.309  1.824  1.521  4.662  1.0

In [10]:
distance_df = pd.read_csv('/Users/kevin/OneDrive - University of Adelaide/MAIML/7209 Big Data Analysis and Project/Robust-Capacity-Expansion-main/InputData/Distances/Distances_NUTS2_DE.csv')
print(distance_df.shape)
print(distance_df.columns)
print(distance_df.head(7))

(38, 39)
Index(['index', 'DE11', 'DE12', 'DE13', 'DE14', 'DE21', 'DE22', 'DE23', 'DE24',
       'DE25', 'DE26', 'DE27', 'DE30', 'DEC0', 'DED2', 'DED4', 'DED5', 'DEE0',
       'DEF0', 'DEG0', 'DE40', 'DE50', 'DE60', 'DE71', 'DE72', 'DE73', 'DE80',
       'DE91', 'DE92', 'DE93', 'DE94', 'DEA1', 'DEA2', 'DEA3', 'DEA4', 'DEA5',
       'DEB1', 'DEB2', 'DEB3'],
      dtype='object')
  index    DE11    DE12    DE13    DE14    DE21    DE22    DE23    DE24  \
0  DE11    0.00   68.99  151.48   99.63  187.90  235.27  187.33  171.22   
1  DE12   68.99    0.00  114.90  115.37  248.73  303.73  255.05  227.24   
2  DE13  151.48  114.90    0.00   95.66  263.20  346.42  324.87  322.50   
3  DE14   99.63  115.37   95.66    0.00  167.97  251.65  238.62  253.95   
4  DE21  187.90  248.73  263.20  167.97    0.00  101.50  145.60  218.45   
5  DE22  235.27  303.73  346.42  251.65  101.50    0.00   90.33  181.67   
6  DE23  187.33  255.05  324.87  238.62  145.60   90.33    0.00   91.43   

     DE25  ...    D

In [11]:
eligibility_df = pd.read_csv('/Users/kevin/OneDrive - University of Adelaide/MAIML/7209 Big Data Analysis and Project/Robust-Capacity-Expansion-main/InputData/Distances/EligibilityNUTS2.csv')
print(eligibility_df.shape)
print(eligibility_df.columns)
print(eligibility_df.head(7))

(38, 39)
Index(['Unnamed: 0', 'DE11', 'DE12', 'DE13', 'DE14', 'DE21', 'DE22', 'DE23',
       'DE24', 'DE25', 'DE26', 'DE27', 'DE30', 'DE40', 'DE50', 'DE60', 'DE71',
       'DE72', 'DE73', 'DE80', 'DE91', 'DE92', 'DE93', 'DE94', 'DEA1', 'DEA2',
       'DEA3', 'DEA4', 'DEA5', 'DEB1', 'DEB2', 'DEB3', 'DEC0', 'DED2', 'DED4',
       'DED5', 'DEE0', 'DEF0', 'DEG0'],
      dtype='object')
  Unnamed: 0  DE11  DE12  DE13  DE14  DE21  DE22  DE23  DE24  DE25  ...  DEB1  \
0       DE11     0     1     0     1     0     0     0     0     1  ...     0   
1       DE12     1     0     1     1     0     0     0     0     0  ...     0   
2       DE13     0     1     0     1     0     0     0     0     0  ...     0   
3       DE14     1     1     1     0     0     0     0     0     0  ...     0   
4       DE21     0     0     0     0     0     1     1     0     1  ...     0   
5       DE22     0     0     0     0     1     0     1     0     0  ...     0   
6       DE23     0     0     0     0     1     1

In [12]:
maxcap_openfieldpv_df = pd.read_csv('/Users/kevin/OneDrive - University of Adelaide/MAIML/7209 Big Data Analysis and Project/Robust-Capacity-Expansion-main/InputData/PV/MaxCap_NUTS2_OpenfieldPV.csv')
print(maxcap_openfieldpv_df.shape)
print(maxcap_openfieldpv_df.columns)
print(maxcap_openfieldpv_df.head(7))

(38, 2)
Index(['Unnamed: 0', 'Cap_max_GW'], dtype='object')
  Unnamed: 0  Cap_max_GW
0       DEF0      17.266
1       DE60       0.386
2       DE91       7.146
3       DE92       9.564
4       DE93      15.981
5       DE94      60.690
6       DE50       0.491


In [13]:
maxcap_rooftoppv_df = pd.read_csv('/Users/kevin/OneDrive - University of Adelaide/MAIML/7209 Big Data Analysis and Project/Robust-Capacity-Expansion-main/InputData/PV/MaxCap_NUTS2_RoofTopPV.csv')
print(maxcap_rooftoppv_df.shape)
print(maxcap_rooftoppv_df.columns)
print(maxcap_rooftoppv_df.head(7))

(38, 2)
Index(['Unnamed: 0', 'Cap_max_GW'], dtype='object')
  Unnamed: 0  Cap_max_GW
0       DEF0      20.521
1       DE60       5.612
2       DE91      10.381
3       DE92      14.010
4       DE93      14.897
5       DE94      24.026
6       DE50       2.833


In [21]:
# PV data processing

data_dir = '/Users/kevin/OneDrive - University of Adelaide/MAIML/7209 Big Data Analysis and Project/Robust-Capacity-Expansion-main/InputData/PV'
years = range(1980, 2020)

dfs = []

for year in years:
    filename = f'PVdata{year}.csv'
    filepath = os.path.join(data_dir, filename)

    df = pd.read_csv(filepath)
    df['Year'] = year

    dfs.append(df)

pv_df = pd.concat(dfs, ignore_index=True)

cols = pv_df.columns.tolist()
cols.insert(0, cols.pop(cols.index('Year')))
pv_df = pv_df[cols]

print(pv_df.shape)
print(pv_df.columns)
print(pv_df.head(5))
print(pv_df.tail(5))


(350400, 40)
Index(['Year', 'Unnamed: 0', 'DE11', 'DE12', 'DE13', 'DE14', 'DE21', 'DE22',
       'DE23', 'DE24', 'DE25', 'DE26', 'DE27', 'DE30', 'DE40', 'DE50', 'DE60',
       'DE71', 'DE72', 'DE73', 'DE80', 'DE91', 'DE92', 'DE93', 'DE94', 'DEA1',
       'DEA2', 'DEA3', 'DEA4', 'DEA5', 'DEB1', 'DEB2', 'DEB3', 'DEC0', 'DED2',
       'DED4', 'DED5', 'DEE0', 'DEF0', 'DEG0'],
      dtype='object')
   Year  Unnamed: 0  DE11  DE12  DE13  DE14  DE21  DE22  DE23  DE24  ...  \
0  1980           0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
1  1980           1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
2  1980           2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
3  1980           3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
4  1980           4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   

   DEB1  DEB2  DEB3  DEC0  DED2  DED4  DED5  DEE0  DEF0  DEG0  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
1   0.0   0.0   0.

In [20]:
maxcap_windoffshore_df = pd.read_csv('/Users/kevin/OneDrive - University of Adelaide/MAIML/7209 Big Data Analysis and Project/Robust-Capacity-Expansion-main/InputData/WindOffshore/MaxCap_NUTS2_WindOffshore.csv')
print(maxcap_windoffshore_df.shape)
print(maxcap_windoffshore_df.columns)
print(maxcap_windoffshore_df.head(7))

(38, 2)
Index(['Unnamed: 0', 'Cap_max_GW'], dtype='object')
  Unnamed: 0  Cap_max_GW
0       DEF0       12.65
1       DE60        0.00
2       DE91        0.00
3       DE92        0.00
4       DE93       12.65
5       DE94       12.65
6       DE50       12.65


In [22]:
# WindOffshore data processing

data_dir = '/Users/kevin/OneDrive - University of Adelaide/MAIML/7209 Big Data Analysis and Project/Robust-Capacity-Expansion-main/InputData/WindOffshore'
years = range(1980, 2020)

dfs = []

for year in years:
    filename = f'OffshoreData{year}.csv'
    filepath = os.path.join(data_dir, filename)

    df = pd.read_csv(filepath)
    df['Year'] = year

    dfs.append(df)

windoffshore_df = pd.concat(dfs, ignore_index=True)

cols = windoffshore_df.columns.tolist()
cols.insert(0, cols.pop(cols.index('Year')))
windoffshore_df = windoffshore_df[cols]

print(windoffshore_df.shape)
print(windoffshore_df.columns)
print(windoffshore_df.head(5))
print(windoffshore_df.tail(5))


(350400, 40)
Index(['Year', 'Unnamed: 0', 'DEF0', 'DE60', 'DE91', 'DE92', 'DE93', 'DE94',
       'DE50', 'DEA1', 'DEA2', 'DEA3', 'DEA4', 'DEA5', 'DE71', 'DE72', 'DE73',
       'DEB1', 'DEB2', 'DEB3', 'DE11', 'DE12', 'DE13', 'DE14', 'DE21', 'DE22',
       'DE23', 'DE24', 'DE25', 'DE26', 'DE27', 'DEC0', 'DE30', 'DE40', 'DE80',
       'DED4', 'DED2', 'DED5', 'DEE0', 'DEG0'],
      dtype='object')
   Year  Unnamed: 0   DEF0   DE60   DE91   DE92   DE93   DE94   DE50   DEA1  \
0  1980           0  0.425  0.425  0.425  0.425  0.425  0.425  0.425  0.425   
1  1980           1  0.421  0.421  0.421  0.421  0.421  0.421  0.421  0.421   
2  1980           2  0.418  0.418  0.418  0.418  0.418  0.418  0.418  0.418   
3  1980           3  0.423  0.423  0.423  0.423  0.423  0.423  0.423  0.423   
4  1980           4  0.422  0.422  0.422  0.422  0.422  0.422  0.422  0.422   

   ...   DE27   DEC0   DE30   DE40   DE80   DED4   DED2   DED5   DEE0   DEG0  
0  ...  0.425  0.425  0.425  0.425  0.425  0.425 

In [23]:
# WindOnshore data processing

data_dir = '/Users/kevin/OneDrive - University of Adelaide/MAIML/7209 Big Data Analysis and Project/Robust-Capacity-Expansion-main/InputData/WindOnshore'
years = range(1980, 2020)

dfs = []

for year in years:
    filename = f'Winddata{year}.csv'
    filepath = os.path.join(data_dir, filename)

    df = pd.read_csv(filepath)
    df['Year'] = year

    dfs.append(df)

windonshore_df = pd.concat(dfs, ignore_index=True)

cols = windonshore_df.columns.tolist()
cols.insert(0, cols.pop(cols.index('Year')))
windonshore_df = windonshore_df[cols]

print(windonshore_df.shape)
print(windonshore_df.columns)
print(windonshore_df.head(5))
print(windonshore_df.tail(5))

(350400, 40)
Index(['Year', 'Unnamed: 0', 'DE11', 'DE12', 'DE13', 'DE14', 'DE21', 'DE22',
       'DE23', 'DE24', 'DE25', 'DE26', 'DE27', 'DE30', 'DE40', 'DE50', 'DE60',
       'DE71', 'DE72', 'DE73', 'DE80', 'DE91', 'DE92', 'DE93', 'DE94', 'DEA1',
       'DEA2', 'DEA3', 'DEA4', 'DEA5', 'DEB1', 'DEB2', 'DEB3', 'DEC0', 'DED2',
       'DED4', 'DED5', 'DEE0', 'DEF0', 'DEG0'],
      dtype='object')
   Year  Unnamed: 0   DE11   DE12   DE13   DE14   DE21   DE22   DE23   DE24  \
0  1980           0  0.310  0.298  0.141  0.396  0.427  0.376  0.345  0.244   
1  1980           1  0.297  0.268  0.108  0.376  0.488  0.390  0.371  0.272   
2  1980           2  0.335  0.266  0.102  0.383  0.507  0.445  0.400  0.299   
3  1980           3  0.344  0.276  0.100  0.391  0.507  0.432  0.411  0.319   
4  1980           4  0.315  0.262  0.094  0.390  0.473  0.390  0.426  0.342   

   ...   DEB1   DEB2   DEB3   DEC0   DED2   DED4   DED5   DEE0   DEF0   DEG0  
0  ...  0.282  0.207  0.279  0.169  0.405  0.364 

In [25]:
maxcap_hydrogencaverns_df = pd.read_csv('/Users/kevin/OneDrive - University of Adelaide/MAIML/7209 Big Data Analysis and Project/Robust-Capacity-Expansion-main/InputData/Salt Caverns/Hydrogencaverns_NUTS2_maxCap.csv')
print(maxcap_hydrogencaverns_df.shape)
print(maxcap_hydrogencaverns_df.columns)
print(maxcap_hydrogencaverns_df.head())

(1, 38)
Index(['DE40', 'DE50', 'DE60', 'DE80', 'DE91', 'DE92', 'DE93', 'DE94', 'DEE0',
       'DEF0', 'DEA1', 'DEA2', 'DEA3', 'DEA4', 'DEA5', 'DE71', 'DE72', 'DE73',
       'DEB1', 'DEB2', 'DEB3', 'DE11', 'DE12', 'DE13', 'DE14', 'DE21', 'DE22',
       'DE23', 'DE24', 'DE25', 'DE26', 'DE27', 'DEC0', 'DE30', 'DED4', 'DED2',
       'DED5', 'DEG0'],
      dtype='object')
      DE40     DE50     DE60     DE80     DE91     DE92     DE93     DE94  \
0  3000000  3000000  3000000  3000000  3000000  3000000  3000000  3000000   

      DEE0     DEF0  ...  DE24  DE25  DE26  DE27  DEC0  DE30  DED4  DED2  \
0  3000000  3000000  ...     0     0     0     0     0     0     0     0   

   DED5  DEG0  
0     0     0  

[1 rows x 38 columns]
